In [1]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.3-py3-none-any.whl (222 kB)

In [ ]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
model_en = fasttext.load_model('cc.en.300.bin')

In [ ]:
model_en.get_nearest_neighbors('good')

[(0.7517593502998352, 'bad'),
 (0.7426098585128784, 'great'),
 (0.7299689054489136, 'decent'),
 (0.7123614549636841, 'nice'),
 (0.6796907186508179, 'Good'),
 (0.6737031936645508, 'excellent'),
 (0.669592022895813, 'goood'),
 (0.6602178812026978, 'ggod'),
 (0.6479219794273376, 'semi-good'),
 (0.6417751908302307, 'good.Good')]

In [ ]:
model_en.get_word_vector("good").shape

(300,)

In [ ]:
model_en.get_analogies("berlin","germany","france")

[(0.7303731441497803, 'paris'),
 (0.6408537030220032, 'france.'),
 (0.6393311023712158, 'avignon'),
 (0.6316676139831543, 'paris.'),
 (0.5895596742630005, 'montpellier'),
 (0.5884554386138916, 'rennes'),
 (0.5850598812103271, 'grenoble'),
 (0.5832924246788025, 'london'),
 (0.5806092619895935, 'strasbourg'),
 (0.574320375919342, 'Paris.')]

In [ ]:
model_en.get_analogies("berlin","germany","india")

[(0.7148876190185547, 'delhi'),
 (0.6974374055862427, 'mumbai'),
 (0.648612916469574, 'jaipur'),
 (0.6349966526031494, 'kolkata'),
 (0.6279922723770142, 'pune'),
 (0.6277596354484558, 'bangalore'),
 (0.6044078469276428, 'hyderabad'),
 (0.6021745800971985, 'noida'),
 (0.6018899083137512, 'bhubaneswar'),
 (0.599077582359314, 'nashik')]

In [ ]:
import pandas as pd

df= pd.read_csv("/content/ecommerceDataset.csv", names=["category", "description"], header=None)
print(df.shape)
df.head(3)

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...


In [ ]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [ ]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [ ]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

In [ ]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

In [ ]:
df['category'] = '__label__' + df['category'].astype(str)
df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [ ]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head(3)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...


In [ ]:
import re
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [ ]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [ ]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [ ]:
train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)

In [ ]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

(10082, 0.9665740924419758, 0.9665740924419758)

In [ ]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99644136]))

In [ ]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [ ]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000978]))

In [ ]:
model.get_nearest_neighbors("painting")

[(0.9986072182655334, 'burnouts'),
 (0.9986058473587036, '297383'),
 (0.9986058473587036, '317'),
 (0.9986043572425842, '1182'),
 (0.9986043572425842, 'cyclopentane'),
 (0.9985969066619873, 'lddr20'),
 (0.9985954165458679, 'conditioned'),
 (0.9985938668251038, 'sodas'),
 (0.9985938668251038, 'incite'),
 (0.9985863566398621, 'tumbling')]

In [ ]:
model.get_nearest_neighbors("sony")

[(0.9992963671684265, 'osx'),
 (0.9992957711219788, 'airbag'),
 (0.9992954134941101, 'wc50'),
 (0.9992890357971191, 'mjx'),
 (0.9992890357971191, 'queued'),
 (0.9992890357971191, 'msata'),
 (0.9992809295654297, 'ip66'),
 (0.9992712140083313, 'lintless'),
 (0.9992693662643433, 'aspirations'),
 (0.9992680549621582, "we'd")]

https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification
